In [1]:
#-*-coding:utf-8-*-
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import MySQLdb as mdb
import time
from pandas import Series,DataFrame
import pandas as pd

In [2]:
db = mdb.connect('localhost','root','','db',charset='utf8')
cursor = db.cursor()

In [3]:
# sql = "truncate douban_user_meiwen_commit_copy "#危险
# cursor.execute(sql)

In [4]:
headers = {
        'User - Agent': 'Mozilla / 5.0(Windows NT 10.0;WOW64) AppleWebKit / 537.36(KHTML, likeGecko) Chrome / 54.0.2840.59Safari / 537.36',
                        #'Cookie':'bid=UhMt-DNW25k; gr_user_id=8571d2c3-dbf8-4a3e-af7d-d4db04dbfcb2; ll="108288"; _ga=GA1.2.1473608639.1474423313; ct=y; ps=y; __utmt=1; __utmt_douban=1; _pk_ref.100001.3ac3=%5B%22%22%2C%22%22%2C1481167434%2C%22https%3A%2F%2Fwww.douban.com%2F%22%5D; viewed="26901394_26879486_1770782_26891290_26880812_26897884_26871656_26821461_26864984_26863797"; gr_cs1_7cb0c3b0-029e-4e0a-9191-3ed8bd1409ec=user_id%3A0; ap=1; dbcl2="140255454:Nmrl1AXqArQ"; ck=e2j3; gr_session_id_22c937bbd8ebd703f2d8e9445f7dfd03=fa5b2f85-dfa4-4786-9a3c-83cf2ac19511; gr_cs1_fa5b2f85-dfa4-4786-9a3c-83cf2ac19511=user_id%3A1; __utma=30149280.1473608639.1474423313.1481006163.1481167432.69; __utmb=30149280.5.10.1481167432; __utmc=30149280; __utmz=30149280.1480555564.60.22.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); __utmv=30149280.14025; __utma=81379588.555128307.1474423313.1481006193.1481167433.57; __utmb=81379588.4.10.1481167433; __utmc=81379588; __utmz=81379588.1481167433.57.19.utmcsr=douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/; _pk_id.100001.3ac3=a7a630736264e063.1474423313.57.1481167484.1481006219.; _pk_ses.100001.3ac3=*; _vwo_uuid_v2=350C9588FEA63E39464BBDDF78D06877|5236f74b68a7e40b1c102a0727ee6f33; push_noty_num=0; push_doumail_num=0'
        # 'Cookie':'bid=UhMt-DNW25k; gr_user_id=8571d2c3-dbf8-4a3e-af7d-d4db04dbfcb2; ll="108288"; viewed="26901394_26879486_1770782_26891290_26880812_26897884_26871656_26821461_26864984_26863797"; ue="13521106693@163.com"; ap=1; _pk_ref.100001.8cb4=%5B%22%22%2C%22%22%2C1484012761%2C%22https%3A%2F%2Fbook.douban.com%2Fsubject%2F6082808%2Freviews%22%5D; _gat_UA-7019765-1=1; _ga=GA1.2.1473608639.1474423313; ps=y; dbcl2="155365448:o56VIJhmz5E"; ck=duEZ; __utmt=1; _vwo_uuid_v2=350C9588FEA63E39464BBDDF78D06877|5236f74b68a7e40b1c102a0727ee6f33; _pk_id.100001.8cb4=c52aef3f07151c32.1474448112.40.1484012815.1483932035.; _pk_ses.100001.8cb4=*; __utma=30149280.1473608639.1474423313.1483949884.1484012813.82; __utmb=30149280.4.10.1484012813; __utmc=30149280; __utmz=30149280.1484012813.82.27.utmcsr=accounts.douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/login; __utmv=30149280.15536; ct=y; push_noty_num=0; push_doumail_num=0'
    }

In [76]:

def main():
    db = mdb.connect('localhost', 'root', '', 'db', charset='utf8')
    cursor = db.cursor()
    count = 0
    books_id = pd.read_csv('douban_all_book_id.csv').book_id
    for book_id in books_id:
        print book_id
        print 'now the total book is:%s' % count
        count += 1
        url = 'https://book.douban.com/subject/%s/reviews' % book_id
        r = requests.get(url, headers=headers)
        if r.status_code==400:
            break
        print r.status_code
        soup = BeautifulSoup(r.text, 'html5lib')
        users_meiwen_id = re.findall('id="toggle-(\d*?)"', r.text)
        title = re.findall('<a href="https://book.douban.com/review/.*/" class="title-link">(.*?)</a>', r.text)
        if users_meiwen_id == []:
            continue
        user_id = re.findall('a href="https://www.douban.com/people/(.*?)/"', r.text)
        user_img = map(lambda x: x.img['src'], soup.find_all('a', attrs={'class': "author-avatar"}))
        user_score_list = re.findall('class="allstar(.*?) main-title-rating"', r.text)
        user_score = ['' if i == [] else i for i in user_score_list]
        time.sleep(5)
        for user_id, user_score, user_img, user_meiwen_id,title in zip(user_id, user_score, user_img, users_meiwen_id,title):
            meiwen_url = 'https://book.douban.com/j/review/%s/full' % user_meiwen_id
            r = requests.get(meiwen_url, headers=headers)
            meiwen = r.json()[u'html'].replace('&nbsp;', ' ').replace(u'豆瓣', '').replace(u"'", "//").replace(u'<br>',u'\n')
            try:
                sql = "INSERT IGNORE INTO douban_user_meiwen_commit_copy(book_id,user_id,user_score,user_img,user_meiwen_commit,title)VALUES('%s','%s','%s','%s','%s','%s')" % (
                book_id, user_id, user_score, user_img, meiwen,title)
                cursor.execute(sql)
                db.commit()
            except:
                print 'an erro in uid:%s  occured when insert data to douban_user_meiwen_commit' % user_id
                db.rollback()
            time.sleep(1)
main()

1000001
now the total book is:0
200
1000019
now the total book is:1
200
1000041
now the total book is:2
200
1000083
now the total book is:3
200
1000093
now the total book is:4
200
1000132
now the total book is:5
200
1000135
now the total book is:6
200
1000144
now the total book is:7
200
1000166
now the total book is:8
200
1000180
now the total book is:9
200
1000188
now the total book is:10
200
1000210
now the total book is:11
200
1000213
now the total book is:12
200
1000216
now the total book is:13
200
1000242
now the total book is:14
200
1000251
now the total book is:15
200
1000280
now the total book is:16
200
1000284
now the total book is:17
200
1000303
now the total book is:18
200
1000307
now the total book is:19
200
1000323
now the total book is:20
200


KeyboardInterrupt: 